In [1]:
# To hold the code of a blank transformer model
import sys
sys.path.append('/workspace/fourth_year_project/MusicGen')
#print(sys.path)
from MyAudioDataset import MyAudioDataset
import torch
from torch import nn
import torchaudio
mydataset = MyAudioDataset('/workspace/small_model_data2', 'recording_01_')
from audiocraft.models import CompressionModel
from audiocraft.models.encodec import InterleaveStereoCompressionModel
model = CompressionModel.get_pretrained('facebook/encodec_32khz')
#model = model.cuda()
comp_model = InterleaveStereoCompressionModel(model)
# move to GPU
##comp_model = comp_model.cuda()

source_comp=model
target_comp=comp_model
#d_model=1500 # once we get more memory
d_model=512
nhead=4
num_layers=4
dim_feedforward=512



/workspace/venv_work/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
#nn.Transformer(d_model, nhead, num_layers, dim_feedforward)

In [12]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward):
        super(SimpleModel, self).__init__()
        self.tl = nn.Transformer(d_model, nhead, num_layers, dim_feedforward) 
        self.il = nn.Linear(1500, d_model)
        self.ol = nn.Linear(d_model, 1500)
    
    def forward(self, audio, angle):
        #audio = audio.cuda()
        print(audio.shape)
        audio = self.il(audio)
        print(audio.shape)

        #angle = self.angle_encoding(angle)  # Process one-hot encoded angle
        #angle = angle.unsqueeze(1).repeat(1, audio.size(2), 1)  # Repeat angle for each time step
        #x = audio + angle  # Combine audio and angle

        #x = self.tl(x.permute(2, 0, 1))
        #x = x.permute(1, 2, 0)
        x = self.ol(audio)
        print(x.shape)
        #print(x)

        #return x.view(-1, 2, 30*44100)
        return x
    
# torch.Size([1, 4, 1500])
# torch.Size([1, 4, 256])
# torch.Size([1, 4, 2])
# tensor([[[-219.0613,  256.4537],
#          [-631.1140,  370.1495],
#          [-156.5977,   39.5030],
#          [  52.4517,  273.8931]]], grad_fn=<ViewBackward0>)


In [13]:
big_model = SimpleModel(d_model, nhead, num_layers, dim_feedforward)

/workspace/venv_work/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [14]:
def compress_mono(mono):
        #mono = mono.cuda()
        with torch.no_grad():
            mono, scale = target_comp.encode(mono)
        return mono

def compress_stereo(stereo):
        #stereo = stereo.cuda()
        with torch.no_grad():
            stereo, scale = target_comp.encode(stereo)
        return stereo

In [15]:
batch_size = 1
lr = 0.0001
loss_fn = nn.CosineEmbeddingLoss()
optimizer = torch.optim.Adam(big_model.parameters(), lr=lr)
train_loader = torch.utils.data.DataLoader(mydataset, batch_size=batch_size, shuffle=True)

In [16]:
target, orig, angle, sr  = train_loader.__iter__().__next__()

In [17]:
# optimizer.zero_grad()
# target = target.cuda()
# orig = orig.cuda()

In [18]:
# Convert wav to codes
target_codes = compress_stereo(target)
orig_codes = compress_mono(orig)
# print(type(target_codes[0][0]))
# print(type(orig_codes[0][0]))

In [19]:
print(orig_codes.shape)

torch.Size([1, 8, 1500])


In [20]:
# Pass codes to model
output = big_model(audio=orig_codes.float(), angle=angle)

torch.Size([1, 8, 1500])
torch.Size([1, 8, 512])
torch.Size([1, 8, 1500])


In [22]:
output.shape

torch.Size([1, 8, 1500])

In [27]:
# loss = loss_fn(output, target_codes)
y = torch.ones(target_codes.size(0))
loss = loss_fn(output.squeeze(0), target_codes.squeeze(0), y)


In [28]:
loss.backward()


In [31]:
epoch = 1
i=1
optimizer.step()
print(f"Epoch {epoch}, batch {i}, loss {loss.item()}")

Epoch 1, batch 1, loss 0.979752779006958
